In [178]:
import duckdb
import pandas as pd
from fredapi import Fred
import certifi
import os
from datetime import datetime, timedelta, date
from config import FRED_API_KEY

os.environ['SSL_CERT_FILE'] = certifi.where() #API Key security requirements
fred = Fred(api_key=FRED_API_KEY)
today_date = date.today().strftime('%Y-%m-%d') #Fetch Date
# List of FRED series IDs
series_ids = ['GDP', 
              'GDPC1', 
              'CORESTICKM159SFRBATL', 
              'WM2NS', 
              'UNRATE', 
              'DFF', 
              'DGS10', 
              'PCE', 
              'INDPRO', 
              'MRTSSM44000USS',
              'PPIACO',
              'HOUST']

### Fetch Data from the Economic Research Federal Reserve Bank of St. Louis

In [179]:
def fetch_series_data(series_id, start_date, end_date):
    """
    Fetch series data for a given series ID between start_date and end_date.
    
    Parameters:
    - series_id (str): The FRED series ID.
    - start_date (str): The start date for data retrieval.
    - end_date (str): The end date for data retrieval.
    
    Returns:
    - pd.Series: The time series data.
    """
    try:
        series_data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
        return series_data
    except Exception as e:
        print(f"Error fetching series data for {series_id}: {e}")
        return pd.Series([])

def fetch_metadata(series_id):
    """
    Fetch metadata for a given series ID.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - dict: The metadata for the series.
    """
    try:
        metadata = fred.get_series_info(series_id)
        return metadata
    except Exception as e:
        print(f"Error fetching metadata for {series_id}: {e}")
        return {}

def construct_dataframe(series_data, metadata, series_id, fetch_date):
    """
    Construct a DataFrame from series data and metadata.
    
    Parameters:
    - series_data (pd.Series): The time series data.
    - metadata (dict): The metadata for the series.
    - series_id (str): The FRED series ID.
    - fetch_date (str): The date when the data was fetched.
    
    Returns:
    - pd.DataFrame: The constructed DataFrame with data and metadata.
    """
    try:
        df = pd.DataFrame(series_data, columns=['Index']).reset_index().rename(columns={'index': 'Date'})
        df['Name'] = metadata.get('title', '')
        df['SeriesID'] = series_id
        df['Source'] = f'https://fred.stlouisfed.org/series/{series_id}'
        df['Units'] = metadata.get('units', '')
        df['SeasonalAdjustment'] = metadata.get('seasonal_adjustment', '')
        df['Frequency'] = metadata.get('frequency', '')
        df['FREDLastUpdatedDate'] = metadata.get('last_updated', '')
        df['FetchDate'] = fetch_date
        return df
    except Exception as e:
        print(f"Error constructing dataframe for {series_id}: {e}")
        return pd.DataFrame()

def fetch_fred_data(series_ids):
    """
    Fetch data and metadata for a list of FRED series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    
    Returns:
    - list of pd.DataFrame: A list of DataFrames with data and metadata for each series.
    """
    today_date = date.today().strftime('%Y-%m-%d')
    twenty_years_ago = (date.today() - timedelta(days=20*365)).strftime('%Y-%m-%d')
    dataframes = []
    
    for series_id in series_ids:
        series_data = fetch_series_data(series_id, twenty_years_ago, today_date)
        if series_data.empty:
            continue
        metadata = fetch_metadata(series_id)
        df = construct_dataframe(series_data, metadata, series_id, today_date)
        dataframes.append(df)
    
    return dataframes


In [180]:
# Fetch data
dataframes = fetch_fred_data(series_ids)

# Display the fetched DataFrames
for df in dataframes:
    display(df.head(2))

,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-01-01,11923.447,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
1,2004-04-01,12112.815,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-01-01,15248.68,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
1,2004-04-01,15366.85,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,2.126567,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
1,2004-04-01,2.247883,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-22,6169.5,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
1,2004-03-29,6167.0,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,5.8,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
1,2004-04-01,5.6,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13


...

,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,8098.8,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
1,2004-04-01,8107.2,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,92.5325,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
1,2004-04-01,92.9096,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,286209.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
1,2004-04-01,282952.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,143.1,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
1,2004-04-01,144.8,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,1998.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
1,2004-04-01,2003.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13


### Connect to DuckDB and Iterate and Create Tables

In [181]:
# Creates empty database and shows existing tables
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: 
    display(con.sql(sql_query).df())

,name


In [182]:
conn = duckdb.connect('data/fred.db')
for series_id, dataframe in zip(series_ids, dataframes):
    # Convert series ID to lowercase for the table name
    table_name = series_id.lower()
    
    # Register the DataFrame as a virtual table
    conn.register(table_name + "_temp", dataframe)
    
    # Create or replace the physical table from the virtual table
    conn.execute(f'''
    CREATE OR REPLACE TABLE {table_name} AS 
    SELECT * FROM {table_name}_temp
    ''')

    # Optionally, unregister the virtual table to clean up
    conn.unregister(table_name + "_temp")
conn.close()

sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: 
    display(con.sql(sql_query).df())

,name
0,corestickm159sfrbatl
1,dff
2,dgs10
3,gdp
4,gdpc1
5,houst
6,indpro
7,mrtssm44000uss
8,pce
9,ppiaco
